# Task1
Model = All
Optimizer = All
Scaler = All
Imputer = All

Index_X = FSR_for_force, FSR_for_coord
Index_y = force, x_coord, y_coord
Data = Splited by Time

In [1]:
def define_searchspace(trial):
    model = trial.suggest_categorical('model', ['fsr_model.LSTM', 'fsr_model.CNN_LSTM', 'fsr_model.ANN'])
    if model == 'fsr_model.LSTM':
        trial.suggest_categorical('model_args/hidden_size', [8, 16, 32, 64, 128])
        trial.suggest_int('model_args/num_layer', 1, 8)
    elif model == 'fsr_model.CNN_LSTM':
        trial.suggest_categorical('model_args/cnn_hidden_size', [8, 16, 32, 64, 128])
        trial.suggest_categorical('model_args/lstm_hidden_size', [8, 16, 32, 64, 128])
        trial.suggest_int('model_args/cnn_num_layer', 1, 8)
        trial.suggest_int('model_args/lstm_num_layer', 1, 8)
    elif model == 'fsr_model.ANN':
        trial.suggest_categorical('model_args/hidden_size', [8, 16, 32, 64, 128])
        trial.suggest_int('model_args/num_layer', 1, 8)
    trial.suggest_categorical('criterion', ['torch.nn.MSELoss'])
    trial.suggest_categorical('optimizer', [
        'torch.optim.Adam',
        'torch.optim.NAdam',
        'torch.optim.Adagrad',
        'torch.optim.RAdam',
        'torch.optim.SGD',
    ])
    trial.suggest_float('optimizer_args/lr', 1e-5, 1e-1, log=True)
    imputer = trial.suggest_categorical('imputer', ['sklearn.impute.SimpleImputer'])
    if imputer == 'sklearn.impute.SimpleImputer':
        trial.suggest_categorical('imputer_args/strategy', [
            'mean',
            'median',
        ])
    trial.suggest_categorical('scaler', [ 
        'sklearn.preprocessing.StandardScaler',
        'sklearn.preprocessing.MinMaxScaler',
        'sklearn.preprocessing.RobustScaler',
    ])
    return {
        'index_X': ['FSR_for_force', 'FSR_for_coord'],
        'index_y': ['force', 'x_coord', 'y_coord'],
        'data_loader': 'fsr_data.get_index_splited_by_time'
    }

In [2]:
import ray.tune
import ray.air
import ray.air.integrations.wandb
import ray.tune.schedulers
from fsr_trainable import FSR_Trainable
import ray.tune.search
import ray.tune.search.optuna

tuner = ray.tune.Tuner(
    trainable=ray.tune.with_resources(
        FSR_Trainable, {'cpu':2},
    ),
    tune_config=ray.tune.TuneConfig(
        num_samples=1000,
        scheduler=ray.tune.schedulers.ASHAScheduler(
            max_t=100,
            grace_period=1,
            reduction_factor=2,
            brackets=1,
            metric='rmse',
            mode='min',
        ),
        search_alg=ray.tune.search.optuna.OptunaSearch(
            space=define_searchspace,
            metric='rmse',
            mode='min',
        ),
    ), 
    run_config=ray.air.RunConfig(
        callbacks=[
            ray.air.integrations.wandb.WandbLoggerCallback(project='FSR-prediction'),
        ],
        checkpoint_config=ray.air.CheckpointConfig(
            num_to_keep=3,
            checkpoint_score_attribute='rmse',
            checkpoint_score_order='min',
            checkpoint_frequency=5,
            checkpoint_at_end=True,
        ),
    ),
)
results = tuner.fit()

[I 2023-07-04 20:42:59,719] A new study created in memory with name: optuna
2023-07-04 20:43:01,836	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2023-07-04 20:43:03,184	INFO tune.py:226 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `Tuner(...)`.


2023-07-04 20:43:03,229	INFO wandb.py:320 -- Already logged into W&B.


Trial name,date,done,hostname,iterations_since_restore,mae,mape,node_ip,pid,rmse,time_since_restore,time_this_iter_s,time_total_s,timestamp,training_iteration,trial_id
FSR_Trainable_080964f7,2023-07-04_20-45-00,True,DESKTOP-0P789CI,2,87.88925355887365,258477813.18996572,172.26.215.93,1282179,253.11161678918478,3.7095401287078857,1.5254478454589844,3.7095401287078857,1688471100,2,080964f7
FSR_Trainable_0bb9589b,2023-07-04_20-43-48,True,DESKTOP-0P789CI,1,103.33253812863518,1.2011708262964786e+17,172.26.215.93,1280791,303.549719722094,4.557123899459839,4.557123899459839,4.557123899459839,1688471028,1,0bb9589b
FSR_Trainable_142de23b,2023-07-04_20-44-26,True,DESKTOP-0P789CI,100,63.99897568372405,195730699.13554934,172.26.215.93,1280118,187.46875582098357,56.30327796936035,0.7838723659515381,56.30327796936035,1688471066,100,142de23b
FSR_Trainable_158a9b24,2023-07-04_20-51-19,True,DESKTOP-0P789CI,1,78.1401118598211,1667884911386210.2,172.26.215.93,1287516,273.071486715629,1.3225984573364258,1.3225984573364258,1.3225984573364258,1688471479,1,158a9b24
FSR_Trainable_186e250c,2023-07-04_20-48-14,True,DESKTOP-0P789CI,4,62.14826543167562,200944916.6501046,172.26.215.93,1285525,198.56037283933927,3.327568769454956,0.7832024097442627,3.327568769454956,1688471294,4,186e250c
FSR_Trainable_321e144e,2023-07-04_20-49-13,True,DESKTOP-0P789CI,100,34.71763334577879,58136711.54522443,172.26.215.93,1283947,116.11913454059426,127.51993680000305,1.3507211208343506,127.51993680000305,1688471353,100,321e144e
FSR_Trainable_35e41ca1,2023-07-04_20-47-11,,DESKTOP-0P789CI,,,,172.26.215.93,1284386,,,,,1688471231,,35e41ca1
FSR_Trainable_3b4317a1,2023-07-04_20-51-17,True,DESKTOP-0P789CI,100,34.174464503710645,56849282.711112,172.26.215.93,1286781,115.72935052730104,56.28534007072449,0.41617798805236816,56.28534007072449,1688471477,100,3b4317a1
FSR_Trainable_3bccdca1,2023-07-04_20-44-31,True,DESKTOP-0P789CI,2,88.58957719991976,1.3420464201248728e+17,172.26.215.93,1281003,265.73623273999453,34.87538170814514,16.770854473114014,34.87538170814514,1688471071,2,3bccdca1
FSR_Trainable_3d14e8cc,2023-07-04_20-45-05,True,DESKTOP-0P789CI,2,86.22325217411463,238614097.20951423,172.26.215.93,1282285,260.90629425253775,1.820894718170166,0.702256441116333,1.820894718170166,1688471105,2,3d14e8cc


(_WandbLoggingActor pid=1280117) wandb: Currently logged in as: seokjin. Use `wandb login --relogin` to force relogin
(_WandbLoggingActor pid=1280117) wandb: Tracking run with wandb version 0.15.4
(_WandbLoggingActor pid=1280117) wandb: Run data is saved locally in /home/seokj/ray_results/FSR_Trainable_2023-07-04_20-42-59/FSR_Trainable_a9cd7e4c_1_criterion=torch_nn_MSELoss,data_loader=fsr_data_get_index_splited_by_time,imputer=sklearn_impute_SimpleIm_2023-07-04_20-43-03/wandb/run-20230704_204314-a9cd7e4c
(_WandbLoggingActor pid=1280117) wandb: Run `wandb offline` to turn off syncing.
(_WandbLoggingActor pid=1280117) wandb: Syncing run FSR_Trainable_a9cd7e4c
(_WandbLoggingActor pid=1280117) wandb: ⭐️ View project at https://wandb.ai/seokjin/FSR-prediction
(_WandbLoggingActor pid=1280117) wandb: 🚀 View run at https://wandb.ai/seokjin/FSR-prediction/runs/a9cd7e4c
(_WandbLoggingActor pid=1280117) wandb: Waiting for W&B process to finish... (success).
(_WandbLoggingActor pid=1280290) wandb: